# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd
import scipy.stats as stats

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
# Your code here:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [4]:
# Your code here:
pokemon.Legendary.value_counts()

False    735
True      65
Name: Legendary, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [5]:
# Your code here:
pokemon.groupby('Legendary').Total.agg([np.mean, np.std]).reset_index()

,Legendary,mean,std
0,False,417.213605,106.760417
1,True,637.384615,60.937389


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [6]:
# Your code here:
stat, pvalue = stats.ttest_ind(pokemon[pokemon['Legendary']==True].Total, pokemon[pokemon['Legendary']==False].Total, equal_var= False)
print(f'{stat}, {pvalue}')

25.8335743895517, 9.357954335957446e-47


What do you conclude from this test? Write your conclusions below.

In [7]:
# Your conclusions here:
#Total points are considerable different between Legendary and non legendary pokemon

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [8]:
# Your code here:
pokemon.groupby('Type 1').Total.agg(np.size).reset_index()

,Type 1,Total
0,Bug,69
1,Dark,31
2,Dragon,32
3,Electric,44
4,Fairy,17
5,Fighting,27
6,Fire,52
7,Flying,4
8,Ghost,32
9,Grass,70


Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [9]:
# Your code here:
pokemon['is_water'] = np.where(pokemon['Type 1']=='Water',1,0)
pokemon[pokemon['is_water']==1].Total.agg([np.mean, np.std]).reset_index()

,index,Total
0,mean,430.455357
1,std,113.188266


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [10]:
# Your code here:
stat, pvalue = stats.ttest_ind(pokemon[pokemon['is_water']==1].Total, pokemon[pokemon['is_water']==0].Total)
print(f'{stat}, {pvalue}')

-0.4418547448849676, 0.6587140317488793


Write your conclusion below.

In [10]:
# Your conclusions here:



# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [25]:
# Your code here:
stat, pvalue = stats.ttest_rel(pokemon['Attack'], pokemon['Defense'])
print(f'{stat}, {pvalue}')

4.325566393330478, 1.7140303479358558e-05


Describe the results of the test in the cell below.

In [12]:
# Your conclusions here:
# Means of attack and deffense are significant different

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [19]:
# Your code here:
stat, pvalue = stats.ttest_rel(pokemon['Sp. Atk'], pokemon['Sp. Def'])
print(f'{stat}, {pvalue}')

0.853986188453353, 0.3933685997548122


Describe the results of the test in the cell below.

In [14]:
# Your conclusions here:
# Means of special attack and special defense are significant different

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [27]:
# Your code here:
stat, pvalue = stats.ttest_1samp(pokemon['Attack']-pokemon['Defense'], popmean=0)
print(f'{stat}, {pvalue}')

4.325566393330478, 1.7140303479358558e-05


# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [23]:
# Your code here:
poke = pd.crosstab(pokemon['Legendary'],pokemon['Type 1'] == 'Water')
poke.head()

Type 1,False,True
Legendary,,
False,627,108
True,61,4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [24]:
# Your code here:
chi2, p, dof, ex = stats.chi2_contingency(poke,correction=False)
print(f'{chi2}, {p}, {dof}, {ex}')


3.617455159932616, 0.0571762700873503, 1, [[632.1 102.9]
 [ 55.9   9.1]]


Based on a 95% confidence, should we reject the null hypothesis?

In [19]:
# Your answer here:

Don't reject thw null hypotesis